In [ ]:
#using this to find good parameters for frangi
import os
import cv2
import numpy as np
from skimage.filters import frangi
from matplotlib import pyplot as plt

# Path to the source images
image_dir = "/content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/frontalangio"

# Function to apply Frangi filter with random parameters
def apply_frangi_with_random_params(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Randomly select parameters within a range
    #sigmas = range(np.random.randint(1, 3), np.random.randint(4, 10), np.random.randint(1, 3))
    sigmas = [1,2,3,5]
    # alpha = np.random.uniform(0.1, 0.5)
    #beta = np.random.uniform(0.1, 0.5)
    #gamma = np.random.uniform(5, 10)
    alpha = 0.3
    beta = 0.40
    gamma = 7

    # Apply Frangi filter
    filtered_image = frangi(
        image,
        sigmas=sigmas,
        alpha=alpha,
        beta=beta,
        gamma=gamma,
        black_ridges=True
    )

    return filtered_image, sigmas, alpha, beta, gamma

# Get all image filenames in the directory
image_filenames = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

try:
    while True:
        # Select one random image
        selected_filename = np.random.choice(image_filenames)


        filtered_image, sigmas, alpha, beta, gamma = apply_frangi_with_random_params(
            os.path.join(image_dir, selected_filename)
        )

        # Print the parameters used
        print(f"Selected image: {selected_filename}")
        print(f"Random Frangi parameters: sigmas={list(sigmas)}, alpha={alpha}, beta={beta}, gamma={gamma}")


        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.imshow(cv2.imread(os.path.join(image_dir, selected_filename), cv2.IMREAD_GRAYSCALE), cmap='gray')
        plt.title('Original Image')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(filtered_image, cmap='copper')  # Using the 'copper' colormap for a gold-like appearance
        plt.title('Filtered with Frangi')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

except KeyboardInterrupt:
    print("\nProcess was stopped by user.")


In [ ]:
#frangi and clahe output to screen
import os
import cv2
import numpy as np
from skimage.filters import frangi
from matplotlib import pyplot as plt
from fastai.vision.all import *
from fastai.vision.gan import *

def enhance_image(image:PILImageBW):
    original_image = np.array(image)
    negated_image = cv2.bitwise_not(original_image)
    kernel = np.ones((50, 50), np.uint8)
    top_hat = cv2.morphologyEx(negated_image, cv2.MORPH_TOPHAT, kernel)
    enhanced_image = cv2.subtract(original_image, top_hat)
    enhanced_image = np.clip(enhanced_image, 0, 255)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(enhanced_image)
    enhanced_image_pil = PILImageBW.create(enhanced_image)
    return enhanced_image_pil

# Path to the source images
image_dir = "/content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/frontalangio"

# Frangi filter parameters
frangi_params = {
    'sigmas': [1, 2, 3, 5],
    'alpha': 0.3,
    'beta': 0.40,
    'gamma': 7,
    'black_ridges': True
}


image_filenames = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]


for filename in image_filenames:

    image_path = os.path.join(image_dir, filename)
    original_image = PILImageBW.create(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE))


    enhanced_image_pil = enhance_image(original_image)
    enhanced_image = np.array(enhanced_image_pil)


    filtered_image = frangi(enhanced_image, **frangi_params)


    print(f"Processed image: {filename}")
    print(f"Frangi parameters: {frangi_params}")

    # Plot the original and filtered images
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(np.array(original_image), cmap='gray')
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(filtered_image, cmap='copper')
    plt.title('Enhanced & Filtered Image')
    plt.axis('off')

    plt.tight_layout()
    plt.show()


In [32]:
#saves images
import os
import cv2
import numpy as np
from skimage.filters import frangi
from matplotlib import pyplot as plt
from fastai.vision.all import *
from fastai.vision.gan import *

def enhance_image(image:PILImageBW):
    original_image = np.array(image)
    negated_image = cv2.bitwise_not(original_image)
    kernel = np.ones((50, 50), np.uint8)
    top_hat = cv2.morphologyEx(negated_image, cv2.MORPH_TOPHAT, kernel)
    enhanced_image = cv2.subtract(original_image, top_hat)
    enhanced_image = np.clip(enhanced_image, 0, 255)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(enhanced_image)
    enhanced_image_pil = PILImageBW.create(enhanced_image)
    return enhanced_image_pil


source_dir = "/content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/frontalangio"
dest_dir = "/content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal"


os.makedirs(dest_dir, exist_ok=True)

# Frangi filter parameters
frangi_params = {
    'sigmas': [1, 2, 3, 5],
    'alpha': 0.3,
    'beta': 0.40,
    'gamma': 7,
    'black_ridges': True
}


image_filenames = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]


for filename in image_filenames:

    image_path = os.path.join(source_dir, filename)
    original_image = PILImageBW.create(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE))

    enhanced_image_pil = enhance_image(original_image)
    enhanced_image = np.array(enhanced_image_pil)


    filtered_image = frangi(enhanced_image, **frangi_params)


    save_path = os.path.join(dest_dir, filename)
    cv2.imwrite(save_path, filtered_image)


    print(f"Filtered image saved: {save_path}")


Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/1_a.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/1_b.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/1_c.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/1_d.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/3_a.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/3_b.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/3_c.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/3_d.png
Filtered image saved: /content/drive/MyDrive/2dto3d/Images/dataset/BW_AND_ROTATED/filtered_frontal/4_a.png
Filtered image saved: /content/drive/